<a href="https://colab.research.google.com/github/analluvias/ensemble-methods/blob/master/Facial_Recognition_with_Supervised_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

nossos dados são derivados do dataset "Labeled Faces in the Wild" (LFW). o objetivo será identificar se alguém é arnold Schwarzenegger ou não.

os dados foram pré-processados com PCA

In [171]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [172]:
import pandas as pd

In [173]:
df = pd.read_csv('/content/drive/MyDrive/projeto-mineracao-dados/lfw_arnie_nonarnie.csv')

In [174]:
df.head()

,0,1,2,3,4,5,6,7,8,9,...,141,142,143,144,145,146,147,148,149,Label
0,-2.061987,0.581320,-0.249115,-0.631340,-1.359899,0.751619,-0.029364,1.610456,0.341402,1.286709,...,-0.356844,-0.016488,-0.228473,0.258134,0.046834,0.135742,-0.068297,0.022038,0.090003,1
1,-0.796838,-0.667228,-0.107889,0.019755,-0.686348,0.912779,0.463412,-0.238308,-0.294023,0.215470,...,-0.037243,-0.012105,-0.351285,-0.034968,0.192314,-0.015406,-0.089117,0.023588,-0.019998,1
2,5.376779,1.142695,2.543111,-2.727212,0.272785,-0.972187,1.111221,1.645502,-2.556968,-0.648781,...,0.157441,-0.333875,-0.303720,-0.085975,0.171346,0.128577,-0.118262,0.045881,-0.190158,1
3,7.029235,1.242883,-2.628079,1.224479,-1.141370,-1.620647,0.205890,1.567561,0.736200,0.010782,...,0.051040,-0.068796,0.141841,-0.227999,0.046044,0.013643,-0.125893,0.146396,0.013320,1
4,5.484822,6.752706,-4.291114,1.740412,-1.603087,-1.075175,1.919936,-0.197615,1.030596,1.451936,...,0.034412,0.265141,0.226000,0.032064,-0.113654,0.059126,-0.216803,0.025849,0.020456,1


In [175]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 190 entries, 0 to 189
Columns: 151 entries, 0 to Label
dtypes: float64(150), int64(1)
memory usage: 224.3 KB


In [176]:
df['Label'].value_counts()

,count
Label,
0,150
1,40


para problemas como esse, sem o uso de cnn e nem redes neurais, vamos utilizar os modelos mais recomendados para o problema:

1. regressão logística
2. svm
3. knn

posteriormente, vamos utilizar os métodos de montagem soft voting e staking com logreg

In [177]:
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler

model_reglog = Pipeline([
    ('reglog', LogisticRegression())])
model_svm = Pipeline([
    ('svm', SVC(probability=True))
    ])
model_knn = Pipeline([
    ('knn', KNeighborsClassifier())
    ])

In [178]:
# vamos fazer uma random search para obter os melhores hiperparâmetros

from sklearn.model_selection import RandomizedSearchCV

param_grid = {
    'reglog': {
        'reglog__C': [0.01, 0.1, 0.5, 1, 10, 50],
        'reglog__solver': ['liblinear', 'saga'],
        'reglog__penalty': ['l1', 'l2'],
        'reglog__class_weight': ['balanced']
    },

    'svm': {
        'svm__C': [0.001, 0.01, 0.1, 0.5, 1, 10, 50],
        'svm__kernel': ['linear', 'rbf', 'sigmoid'],
        'svm__gamma': ['scale', 'auto'],
        'svm__class_weight': ['balanced']
    },

    'knn': {
        'knn__n_neighbors': [3, 5, 7, 9, 11, 13, 15],
        'knn__weights': ['uniform', 'distance'],
        'knn__p': [1, 2]
    }
}

In [179]:
random_search_reglog = RandomizedSearchCV(
    model_reglog,
    param_distributions=param_grid['reglog'],
    n_iter=25,
    cv=5,
    scoring='f1',
    n_jobs=-1,
)

In [180]:
random_search_svm = RandomizedSearchCV(
    model_svm,
    param_distributions=param_grid['svm'],
    n_iter = 25,
    cv=5,
    scoring='f1',
    n_jobs=-1,
)

In [181]:
random_search_knn = RandomizedSearchCV(
    model_knn,
    param_distributions=param_grid['knn'],
    n_iter = 25,
    cv = 5,
    scoring = 'f1',
    n_jobs = -1,
)

separando os dados em treino e teste

In [182]:
from sklearn.model_selection import train_test_split

X = df.drop('Label', axis=1)
y = df['Label']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

visualizando os melhores scores

In [183]:
random_search_reglog.fit(X_train, y_train)

/usr/local/lib/python3.12/dist-packages/sklearn/model_selection/_search.py:317: UserWarning: The total space of parameters 24 is smaller than n_iter=25. Running 24 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


RandomizedSearchCV(cv=5,
                   estimator=Pipeline(steps=[('reglog', LogisticRegression())]),
                   n_iter=25, n_jobs=-1,
                   param_distributions={'reglog__C': [0.01, 0.1, 0.5, 1, 10,
                                                      50],
                                        'reglog__class_weight': ['balanced'],
                                        'reglog__penalty': ['l1', 'l2'],
                                        'reglog__solver': ['liblinear',
                                                           'saga']},
                   scoring='f1')

In [184]:
print(random_search_reglog.best_params_)
print(random_search_reglog.best_score_)

{'reglog__solver': 'liblinear', 'reglog__penalty': 'l2', 'reglog__class_weight': 'balanced', 'reglog__C': 0.5}
0.6690476190476191


In [185]:
random_search_svm.fit(X_train, y_train)

RandomizedSearchCV(cv=5,
                   estimator=Pipeline(steps=[('svm', SVC(probability=True))]),
                   n_iter=25, n_jobs=-1,
                   param_distributions={'svm__C': [0.001, 0.01, 0.1, 0.5, 1, 10,
                                                   50],
                                        'svm__class_weight': ['balanced'],
                                        'svm__gamma': ['scale', 'auto'],
                                        'svm__kernel': ['linear', 'rbf',
                                                        'sigmoid']},
                   scoring='f1')

In [186]:
print(random_search_svm.best_params_)
print(random_search_svm.best_score_)

{'svm__kernel': 'rbf', 'svm__gamma': 'auto', 'svm__class_weight': 'balanced', 'svm__C': 50}
0.6319480519480519


In [187]:
random_search_knn.fit(X_train, y_train)

RandomizedSearchCV(cv=5,
                   estimator=Pipeline(steps=[('knn', KNeighborsClassifier())]),
                   n_iter=25, n_jobs=-1,
                   param_distributions={'knn__n_neighbors': [3, 5, 7, 9, 11, 13,
                                                             15],
                                        'knn__p': [1, 2],
                                        'knn__weights': ['uniform',
                                                         'distance']},
                   scoring='f1')

In [188]:
print(random_search_knn.best_params_)
print(random_search_knn.best_score_)

{'knn__weights': 'distance', 'knn__p': 2, 'knn__n_neighbors': 3}
0.4564102564102564


vamos agora utilizar ensemble methods

In [189]:
# modelos base

reglog_base = Pipeline([
    ('reglog_base', LogisticRegression(solver= 'liblinear', penalty= 'l2', class_weight= 'balanced', C= 0.5))
])



svm_base = Pipeline([
    ('svm_base', SVC(probability=True, kernel= 'rbf', gamma= 'auto', class_weight= 'balanced', C= 1))
])

knn_base = Pipeline([
    ('knn_base', KNeighborsClassifier(weights= 'distance', p= 2, n_neighbors= 3))
])

In [190]:
# primeiro faremos um soft voting

from sklearn.ensemble import VotingClassifier

voting_clf = Pipeline([
    ("voting", VotingClassifier(
        estimators=[
            ("reglog", reglog_base),
            ("svm", svm_base),
            ("knn", knn_base)
        ],
        voting = 'soft'
    ))

])


In [191]:
voting_clf.fit(X_train, y_train)

Pipeline(steps=[('voting',
                 VotingClassifier(estimators=[('reglog',
                                               Pipeline(steps=[('reglog_base',
                                                                LogisticRegression(C=0.5,
                                                                                   class_weight='balanced',
                                                                                   solver='liblinear'))])),
                                              ('svm',
                                               Pipeline(steps=[('svm_base',
                                                                SVC(C=1,
                                                                    class_weight='balanced',
                                                                    gamma='auto',
                                                                    probability=True))])),
                                              ('knn',
                                               Pipeline(steps=[('knn_base',
                                                                KNeighborsClassifier(n_neighbors=3,
                                                                                     weights='distance'))]))],
                                  voting='soft'))])

In [192]:
best_reglog_params = random_search_reglog.best_params_
best_svm_params = random_search_svm.best_params_
best_knn_params = random_search_knn.best_params_


In [193]:
# vamos agora treinar um stacking
from sklearn.ensemble import StackingClassifier

stacking_clf = StackingClassifier(
    estimators=[
        ("reglog", reglog_base),
        ("svm", svm_base),
        ("knn", knn_base)
    ],
    final_estimator = LogisticRegression(),
    cv=5,                         # obrigatório para gerar preds out-of-fold
    stack_method="predict_proba", # garante uso das probas como features do meta
)


In [194]:
stacking_clf.fit(X_train, y_train)

StackingClassifier(cv=5,
                   estimators=[('reglog',
                                Pipeline(steps=[('reglog_base',
                                                 LogisticRegression(C=0.5,
                                                                    class_weight='balanced',
                                                                    solver='liblinear'))])),
                               ('svm',
                                Pipeline(steps=[('svm_base',
                                                 SVC(C=1,
                                                     class_weight='balanced',
                                                     gamma='auto',
                                                     probability=True))])),
                               ('knn',
                                Pipeline(steps=[('knn_base',
                                                 KNeighborsClassifier(n_neighbors=3,
                                                                      weights='distance'))]))],
                   final_estimator=LogisticRegression(),
                   stack_method='predict_proba')

In [195]:
from sklearn.metrics import classification_report, f1_score, roc_auc_score

def avaliar(modelo, X_test, y_test, nome):
    y_pred = modelo.predict(X_test)
    print(f"\n===== {nome} =====")
    print("f1:", f1_score(y_test, y_pred))
    print("ROC AUC Score:", roc_auc_score(y_test,  modelo.predict_proba(X_test)[:,1]))
    print(classification_report(y_test, y_pred))

In [196]:
avaliar(voting_clf, X_test, y_test, "VotingClassifier")
avaliar(stacking_clf, X_test, y_test, "StackingClassifier")


===== VotingClassifier =====
f1: 0.7058823529411765
ROC AUC Score: 0.9458333333333334
              precision    recall  f1-score   support

           0       0.93      0.90      0.92        30
           1       0.67      0.75      0.71         8

    accuracy                           0.87        38
   macro avg       0.80      0.82      0.81        38
weighted avg       0.88      0.87      0.87        38


===== StackingClassifier =====
f1: 0.6666666666666666
ROC AUC Score: 0.9416666666666667
              precision    recall  f1-score   support

           0       0.90      0.93      0.92        30
           1       0.71      0.62      0.67         8

    accuracy                           0.87        38
   macro avg       0.81      0.78      0.79        38
weighted avg       0.86      0.87      0.87        38



podemos ver que o voting classifier superou todos os modelos simples com uma ótima margem, enquanto o stacking praticamente mantece a pontuação do melhor modelo solo treinado (a reglog)